In [ ]:
# 런타임 다시 시작

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from google.colab import drive
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from sklearn.cluster import KMeans

In [ ]:
drive.mount('/content/drive')
df_merge1 =  pd.read_csv('/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/전처리 데이터/final_merged_data.csv')
df_merge1.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,행정동명,시군구명,행정동명.1,행정동코드,CCTV개수,가로등개수,버스정류소개수,음식점개수,주점개수,주차장개수,지하철역개수,자동차소유대수
0,강남구 개포1동,강남구,개포1동,1168066000,57.0,0.0,15.0,6.0,0.0,30.0,0.0,3875.0
1,강남구 개포2동,강남구,개포2동,1168067000,231.0,0.0,34.0,94.0,8.0,154.0,3.0,10266.0
2,강남구 개포3동,강남구,개포3동,1168067500,108.0,102.0,11.0,43.0,1.0,29.0,1.0,6205.0
3,강남구 개포4동,강남구,개포4동,1168069000,381.0,0.0,28.0,141.0,19.0,1026.0,0.0,8552.0
4,강남구 논현1동,강남구,논현1동,1168052100,478.0,0.0,17.0,548.0,106.0,2136.0,2.0,10388.0


In [ ]:
df_enf = pd.read_csv('/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/전처리 데이터/0전처리 서울 음주사고 데이터.csv')
df_accident = pd.read_csv('/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/전처리 데이터/0행정동코드별 사고건수.csv')

In [ ]:
df_enf.head()
df_enf.rename(columns={'자치구별':'시군구명'}, inplace=True)
df_enf.head()

,시군구명,최근 2년 평균 교통사고건수,차량 1만대당 사고건수,교통위법 총계,음주운전 위법 총계,음주운전 비율,전체 음주율,남자 음주율,여자 음주율,22년 음주단속수
0,강남구,3818,142.85,49234,1428,0.029004,5.8,9.4,2.7,1998
1,강동구,1168,68.30,91023,459,0.005043,6.1,9.4,2.9,524
2,강북구,1008,108.50,48222,521,0.010804,12.6,19.7,6.0,545
3,강서구,1562,69.40,131743,883,0.006702,8.1,13.8,2.8,893
4,관악구,1076,76.75,152993,661,0.004320,10.2,15.7,4.6,703


In [ ]:
# 26번째 행의 1번째 열 값을 '강동구'로 변경
df_merge1.iloc[26, 1] = '강동구'

# 26번째 행의 2번째 열 값을 '둔촌1동'으로 변경
df_merge1.iloc[26, 2] = '둔촌1동'

In [ ]:
# 음주 단속 수 병합
df_merge2 = pd.merge(df_merge1, df_enf[['시군구명', '22년 음주단속수']], on='시군구명', how='left')
df_merge2.head()

,행정동명,시군구명,행정동명.1,행정동코드,CCTV개수,가로등개수,버스정류소개수,음식점개수,주점개수,주차장개수,지하철역개수,자동차소유대수,22년 음주단속수
0,강남구 개포1동,강남구,개포1동,1168066000,57.0,0.0,15.0,6.0,0.0,30.0,0.0,3875.0,1998
1,강남구 개포2동,강남구,개포2동,1168067000,231.0,0.0,34.0,94.0,8.0,154.0,3.0,10266.0,1998
2,강남구 개포3동,강남구,개포3동,1168067500,108.0,102.0,11.0,43.0,1.0,29.0,1.0,6205.0,1998
3,강남구 개포4동,강남구,개포4동,1168069000,381.0,0.0,28.0,141.0,19.0,1026.0,0.0,8552.0,1998
4,강남구 논현1동,강남구,논현1동,1168052100,478.0,0.0,17.0,548.0,106.0,2136.0,2.0,10388.0,1998


In [ ]:
df_merge2[df_merge2['22년 음주단속수'].isnull()]

,행정동명,시군구명,행정동명.1,행정동코드,CCTV개수,가로등개수,버스정류소개수,음식점개수,주점개수,주차장개수,지하철역개수,자동차소유대수,22년 음주단속수


In [ ]:
print(df_merge2.head())
df_accident.head()

       행정동명 시군구명 행정동명.1       행정동코드  CCTV개수  가로등개수  버스정류소개수  음식점개수   주점개수  \
0  강남구 개포1동  강남구   개포1동  1168066000    57.0    0.0     15.0    6.0    0.0   
1  강남구 개포2동  강남구   개포2동  1168067000   231.0    0.0     34.0   94.0    8.0   
2  강남구 개포3동  강남구   개포3동  1168067500   108.0  102.0     11.0   43.0    1.0   
3  강남구 개포4동  강남구   개포4동  1168069000   381.0    0.0     28.0  141.0   19.0   
4  강남구 논현1동  강남구   논현1동  1168052100   478.0    0.0     17.0  548.0  106.0   

    주차장개수  지하철역개수  자동차소유대수  22년 음주단속수  
0    30.0     0.0   3875.0       1998  
1   154.0     3.0  10266.0       1998  
2    29.0     1.0   6205.0       1998  
3  1026.0     0.0   8552.0       1998  
4  2136.0     2.0  10388.0       1998  


,adm_cd2,사고건수
0,1111058000,4
1,1111061500,3
2,1111070000,3
3,1111071000,6
4,1114058000,3


In [ ]:
df_accident.rename(columns={'adm_cd2':'행정동코드'}, inplace=True)
df_accident.head()

,행정동코드,사고건수
0,1111058000,4
1,1111061500,3
2,1111070000,3
3,1111071000,6
4,1114058000,3


In [ ]:
# 음주운전 사고 건수 병합
df_merge3 = pd.merge(df_merge2, df_accident[['행정동코드', '사고건수']], on='행정동코드', how='left')
df_merge3.head()

,행정동명,시군구명,행정동명.1,행정동코드,CCTV개수,가로등개수,버스정류소개수,음식점개수,주점개수,주차장개수,지하철역개수,자동차소유대수,22년 음주단속수,사고건수
0,강남구 개포1동,강남구,개포1동,1168066000,57.0,0.0,15.0,6.0,0.0,30.0,0.0,3875.0,1998,NaN
1,강남구 개포2동,강남구,개포2동,1168067000,231.0,0.0,34.0,94.0,8.0,154.0,3.0,10266.0,1998,3.0
2,강남구 개포3동,강남구,개포3동,1168067500,108.0,102.0,11.0,43.0,1.0,29.0,1.0,6205.0,1998,NaN
3,강남구 개포4동,강남구,개포4동,1168069000,381.0,0.0,28.0,141.0,19.0,1026.0,0.0,8552.0,1998,NaN
4,강남구 논현1동,강남구,논현1동,1168052100,478.0,0.0,17.0,548.0,106.0,2136.0,2.0,10388.0,1998,15.0


In [ ]:
df_merge3.fillna(0, inplace=True)
df_merge3.head()

,행정동명,시군구명,행정동명.1,행정동코드,CCTV개수,가로등개수,버스정류소개수,음식점개수,주점개수,주차장개수,지하철역개수,자동차소유대수,22년 음주단속수,사고건수
0,강남구 개포1동,강남구,개포1동,1168066000,57.0,0.0,15.0,6.0,0.0,30.0,0.0,3875.0,1998,0.0
1,강남구 개포2동,강남구,개포2동,1168067000,231.0,0.0,34.0,94.0,8.0,154.0,3.0,10266.0,1998,3.0
2,강남구 개포3동,강남구,개포3동,1168067500,108.0,102.0,11.0,43.0,1.0,29.0,1.0,6205.0,1998,0.0
3,강남구 개포4동,강남구,개포4동,1168069000,381.0,0.0,28.0,141.0,19.0,1026.0,0.0,8552.0,1998,0.0
4,강남구 논현1동,강남구,논현1동,1168052100,478.0,0.0,17.0,548.0,106.0,2136.0,2.0,10388.0,1998,15.0


In [ ]:
# 사고 건수 데이터에 가중치 *1000
df_merge3['음주운전_y'] = df_merge3['22년 음주단속수'] + df_merge3['사고건수'] * 1000
df_merge3.head()

,행정동명,시군구명,행정동명.1,행정동코드,CCTV개수,가로등개수,버스정류소개수,음식점개수,주점개수,주차장개수,지하철역개수,자동차소유대수,22년 음주단속수,사고건수,음주운전_y
0,강남구 개포1동,강남구,개포1동,1168066000,57.0,0.0,15.0,6.0,0.0,30.0,0.0,3875.0,1998,0.0,1998.0
1,강남구 개포2동,강남구,개포2동,1168067000,231.0,0.0,34.0,94.0,8.0,154.0,3.0,10266.0,1998,3.0,4998.0
2,강남구 개포3동,강남구,개포3동,1168067500,108.0,102.0,11.0,43.0,1.0,29.0,1.0,6205.0,1998,0.0,1998.0
3,강남구 개포4동,강남구,개포4동,1168069000,381.0,0.0,28.0,141.0,19.0,1026.0,0.0,8552.0,1998,0.0,1998.0
4,강남구 논현1동,강남구,논현1동,1168052100,478.0,0.0,17.0,548.0,106.0,2136.0,2.0,10388.0,1998,15.0,16998.0


In [ ]:
df_merge3.to_csv('/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/전처리 데이터/최종병합데이터_소현.csv')